# To do:
* Scrape listings
* List of Boroughs
* Data on crime rates, salaries etc... (Kaggle, Google, Gov stats)
* Save it all to database

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

# Scraping all boroughs process:
* ~~scrape boroughs into a list~~
    * loop through boroughs list to create urls
        * get number of pages with each url
            * loop through listings with range

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/London_boroughs'
wiki = requests.get(wiki_url)
wiki_soup = BeautifulSoup(wiki.content, 'html.parser')
ols = wiki_soup.select('td > ol')
boroughs = []
for ol in ols:
    lis = ol.find_all('li')
    for li in lis:
        boroughs.append(li.find('a').getText())
        
# List of boroughs
boroughs = [x.replace(' ', '-').lower() + '-london-borough' for x in boroughs]
# Exceptions treatment
boroughs[boroughs.index('kensington-and-chelsea-london-borough')] = 'kensington-and-chelsea-royal-borough'
boroughs[boroughs.index('city-of-westminster-london-borough')] = 'city-of-westminster'
boroughs[boroughs.index('kingston-upon-thames-london-borough')] = 'kingston-upon-thames'
boroughs[boroughs.index('greenwich-london-borough')] = 'greenwich-royal-borough'
boroughs

['city-of-london-london-borough',
 'city-of-westminster',
 'kensington-and-chelsea-royal-borough',
 'hammersmith-and-fulham-london-borough',
 'wandsworth-london-borough',
 'lambeth-london-borough',
 'southwark-london-borough',
 'tower-hamlets-london-borough',
 'hackney-london-borough',
 'islington-london-borough',
 'camden-london-borough',
 'brent-london-borough',
 'ealing-london-borough',
 'hounslow-london-borough',
 'richmond-upon-thames-london-borough',
 'kingston-upon-thames',
 'merton-london-borough',
 'sutton-london-borough',
 'croydon-london-borough',
 'bromley-london-borough',
 'lewisham-london-borough',
 'greenwich-royal-borough',
 'bexley-london-borough',
 'havering-london-borough',
 'barking-and-dagenham-london-borough',
 'redbridge-london-borough',
 'newham-london-borough',
 'waltham-forest-london-borough',
 'haringey-london-borough',
 'enfield-london-borough',
 'barnet-london-borough',
 'harrow-london-borough',
 'hillingdon-london-borough']

In [3]:
ids = []
borough_list = []
price_list = []
bed_list = []
bath_list = []
chair_list = []
link_list = []
title_list = []
address_list = []
transport1_list = []
transport2_list = []
transport_type_1_list = []
transport_type_2_list = []
tag_list = []
parking_list = []

In [4]:
# Creating the links
links = []
pages = []
for borough in boroughs:
    links.append(f'https://www.zoopla.co.uk/for-sale/property/{borough}/?page_size=25&q={borough}&radius=0&results_sort=newest_listings&pn=1')

# Looping through all links and scraoing the listings
for link in links:
    try:
        data = requests.get(link).content
        soup = BeautifulSoup(data, 'html.parser')
        last_page_nr = int(soup.find_all('li', {'class': 'eaoxhri1'})[-1].find('a').getText()) + 1
        pages.append(last_page_nr)
    except:
        print(link)
        
len(links) == len(pages)

True

In [ ]:
for i, borough in enumerate(boroughs):
    print(f'Starting {borough}')
    for n in range(1, pages[i]):        
        try:
            base_url = f'https://www.zoopla.co.uk/for-sale/property/{borough}/?page_size=25&q={borough}&radius=0&results_sort=newest_listings&pn={n}'
            data = requests.get(base_url).content
            soup = BeautifulSoup(data, 'html.parser')
            listings_container = soup.find('div', {'class': 'css-kdnpqc-ListingsContainer'})
            listings = listings_container.find_all('div', {'class': 'e2uk8e18'})
            regex = re.compile(r'search-result_listing_\d+')
            listing_ids = soup.find_all('div', {'data-testid': regex})
            
            #For loop for the listings list
            for i, listing in enumerate(listings):
                try:
                    id_num = listing_ids[i].attrs['id']
                    ids.append(id_num)
                    #Borough
                    borough_list.append(borough)
                    #Price
                    price = listing.find('p', {'class': 'eczcs4p0'}).getText()
                    price_list.append(price)

                    #Features
                    bed = listing.find('span', {'data-testid': 'bed'})
                    bath = listing.find('span', {'data-testid': 'bath'})
                    chair = listing.find('span', {'data-testid': 'chair'})
                    if bed:
                        bed_list.append(bed.find_next('p').getText())
                    else:
                        bed_list.append('1')
                    if bath:
                        bath_list.append(bath.find_next('p').getText())
                    else:
                        bath_list.append('1')
                    if chair:
                        chair_list.append(chair.find_next('p').getText())
                    else:
                        chair_list.append('0')

                    #Links
                    link = listing.find('a', {'data-testid': 'listing-details-link'}).attrs['href']
                    link_list.append(link)

                    #Title
                    title = listing.find('h2', {'class': 'e2uk8e14'}).getText()
                    title_list.append(title)

                    #Address
                    address = listing.find('h2', {'class': 'e2uk8e14'}).find_next('p').getText()
                    address_list.append(address)

                    #Transportation
                    transport_container = listing.find('div', {'class': 'css-braguw-TransportWrapper'}).find_all('div', {'class': 'ejjz7ko0'})
                    transp1, transp2 = transport_container[:2]

                    transport1 = transp1.attrs['content']
                    transport2 = transp2.attrs['content']

                    transport1_list.append(transport1)
                    transport2_list.append(transport2)

                    transport_type_1 = transp1.find('span', {'role': 'presentation'}).attrs['data-testid']
                    transport_type_2 = transp2.find('span', {'role': 'presentation'}).attrs['data-testid']

                    transport_type_1_list.append(transport_type_1)
                    transport_type_2_list.append(transport_type_2)

                    #Parking
                    parking = listing.find('div', {'content': 'Parking'})
                    if parking:
                        parking_list.append(1)
                    else:
                        parking_list.append(0)

                    #Tags
                    tag = listings[0].find('span', {'class': 'css-10ddrfy-Tag-StyledTag'})
                    if tag:
                        tag_list.append(tag.getText())
                    else:
                        tag_list.append('None')
                        
                except:
                    print(f'Problem at {id_num} on page {n} of {borough}')
                    
                    
        
        except Exception as err:
            print(f'{borough}, page {n}, listing {id_num} has an error {str(err)}')
            

    print(f'{borough} has been scraped')
        

Starting city-of-london-london-borough
city-of-london-london-borough has been scraped
Starting city-of-westminster
city-of-westminster, page 199, listing listing_51502921 has an error 'NoneType' object has no attribute 'find_all'
city-of-westminster has been scraped
Starting kensington-and-chelsea-royal-borough
kensington-and-chelsea-royal-borough, page 37, listing listing_58246684 has an error 'NoneType' object has no attribute 'find_all'
kensington-and-chelsea-royal-borough, page 49, listing listing_58090695 has an error 'NoneType' object has no attribute 'find_all'


In [59]:
def page_test(borough, n, id_num):
    base_url = f'https://www.zoopla.co.uk/for-sale/property/{borough}/?page_size=25&q={borough}&radius=0&results_sort=newest_listings&pn={n}'
    print(base_url)
    data = requests.get(base_url).content
    soup = BeautifulSoup(data, 'html.parser')

    listing = soup.find('div', {'id': id_num})
    
    #For loop for the listings list

    try:
        print(id_num)
        #Borough
        print(borough)
        #Price
        price = listing.find('p', {'class': 'eczcs4p0'}).getText()
        print(price)

        #Features
        bed = listing.find('span', {'data-testid': 'bed'})
        bath = listing.find('span', {'data-testid': 'bath'})
        chair = listing.find('span', {'data-testid': 'chair'})
        if bed:
            print(bed.find_next('p').getText())
        else:
            print('1')
        if bath:
            print(bath.find_next('p').getText())
        else:
            print('1')
        if chair:
            print(chair.find_next('p').getText())
        else:
            print('0')

        #Links
        link = listing.find('a', {'data-testid': 'listing-details-link'}).attrs['href']
        print(link)

        #Title
        title = listing.find('h2', {'class': 'e2uk8e14'}).getText()
        print(title)

        #Address
        address = listing.find('h2', {'class': 'e2uk8e14'}).find_next('p').getText()
        print(address)

        #Transportation
        transport_container = listing.find('div', {'class': 'css-braguw-TransportWrapper'}).find_all('div', {'class': 'ejjz7ko0'})
        transp1, transp2 = transport_container[:2]

        transport1 = transp1.attrs['content']
        transport2 = transp2.attrs['content']

        print(transport1)
        print(transport2)

        transport_type_1 = transp1.find('span', {'role': 'presentation'}).attrs['data-testid']
        transport_type_2 = transp2.find('span', {'role': 'presentation'}).attrs['data-testid']

        print(transport_type_1)
        print(transport_type_2)

        #Parking
        parking = listing.find('div', {'content': 'Parking'})
        if parking:
            print(1)
        else:
            print(0)

        #Tags
        tag = listings[0].find('span', {'class': 'css-10ddrfy-Tag-StyledTag'})
        if tag:
            print(tag.getText())
        else:
            print('None')

    except Exception as err:
        print(f'{str(err)}')


In [57]:
page_test('islington-london-borough', 49)